##  기상청 날씨 데이터

https://www.data.go.kr/data/15084084/openapi.do

In [1]:
#!pip install openpyxl
#!pip install requests pandas


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import urllib.parse


SERVICE_KEY = "d88b1ae9ffdae01dc67a2b4073ec4a035567a3a2a436e495b271eaa02b4cc69f"
CITY = "서울특별시"
DISTRICT = "송파구"
EXCEL_PATH = "기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(2411).xlsx"


def get_grid_coords(city, district, path):
    try:
        df = pd.read_excel(path)
        df.columns = df.columns.str.strip()
        loc = df[(df['1단계'] == city) & (df['2단계'] == district)]
        if not loc.empty:
            return loc.iloc[0]['격자 X'], loc.iloc[0]['격자 Y']
        return None, None
    except:
        return None, None


def get_latest_base_time():
    d = datetime.now().strftime('%Y%m%d')
    return d, "0200"


def get_weather_forecast(key, nx, ny):
    url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    base_date, base_time = get_latest_base_time()
    key_enc = urllib.parse.quote_plus(key)

    params = {
        'serviceKey': key_enc,
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'JSON',
        'base_date': base_date,
        'base_time': base_time,
        'nx': str(nx),
        'ny': str(ny)
    }

    try:
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        if data.get('response', {}).get('header', {}).get('resultCode') == '00':
            return data['response']['body']['items']['item']
        return None
    except:
        return None


def create_forecast_dataframe(raw):
    if not raw:
        return pd.DataFrame()

    cat = {
        'POP': '강수확률', 'PTY': '강수형태', 'PCP': '1시간 강수량', 'REH': '습도',
        'SNO': '1시간 신적설', 'SKY': '하늘상태', 'TMP': '1시간 기온',
        'TMN': '일 최저기온', 'TMX': '일 최고기온', 'WSD': '풍속'
    }
    sky = {'1': '맑음', '3': '구름많음', '4': '흐림'}
    pty = {'0': '없음', '1': '비', '2': '비/눈', '3': '눈', '4': '소나기'}

    df = pd.DataFrame(raw)
    df['category'] = df['category'].map(cat)

    df.loc[df['category'] == '하늘상태', 'fcstValue'] = df.loc[df['category'] == '하늘상태', 'fcstValue'].map(sky)
    df.loc[df['category'] == '강수형태', 'fcstValue'] = df.loc[df['category'] == '강수형태', 'fcstValue'].map(pty)

    df2 = df.pivot_table(
        index=['fcstDate', 'fcstTime'],
        columns='category',
        values='fcstValue',
        aggfunc='first'
    ).reset_index()

    df2.rename(columns={'fcstDate': '예보일자', 'fcstTime': '예보시각'}, inplace=True)

    if '일 최저기온' in df2.columns:
        df2['일 최저기온'] = df2.groupby('예보일자')['일 최저기온'].transform(lambda x: x.ffill().bfill())
    if '일 최고기온' in df2.columns:
        df2['일 최고기온'] = df2.groupby('예보일자')['일 최고기온'].transform(lambda x: x.ffill().bfill())

    cols = [
        '예보일자', '예보시각', '1시간 기온', '하늘상태', '습도', '강수확률',
        '강수형태', '1시간 강수량', '풍속', '일 최저기온', '일 최고기온'
    ]
    cols = [c for c in cols if c in df2.columns]

    return df2[cols]


if __name__ == "__main__":
    nx, ny = get_grid_coords(CITY, DISTRICT, EXCEL_PATH)
    if nx and ny:
        raw = get_weather_forecast(SERVICE_KEY, nx, ny)
        if raw:
            df3 = create_forecast_dataframe(raw)
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)


'서울특별시 송파구'의 격자 좌표: nx=62, ny=126
전체 데이터 조회를 위해 예보 발표 시각을 '20251029 0200'으로 고정합니다.


C:\Users\mcdor\AppData\Local\Temp\ipykernel_1424\3429087446.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pivot['일 최저기온'] = df_pivot.groupby('예보일자')['일 최저기온'].transform(lambda x: x.ffill().bfill())
C:\Users\mcdor\AppData\Local\Temp\ipykernel_1424\3429087446.py:119: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pivot['일 최고기온'] = df_pivot.groupby('예보일자')['일 최고기온'].transform(lambda x: x.ffill().bfill())


In [4]:
df3

category,예보일자,예보시각,1시간 기온,하늘상태,습도,강수확률,강수형태,1시간 강수량,풍속,일 최저기온,일 최고기온
0,20251029,0300,6,맑음 ☀️,70,0,없음,강수없음,1,5.0,18.0
1,20251029,0400,6,맑음 ☀️,70,0,없음,강수없음,0.9,5.0,18.0
2,20251029,0500,6,맑음 ☀️,65,0,없음,강수없음,0.9,5.0,18.0
3,20251029,0600,6,맑음 ☀️,70,0,없음,강수없음,1,5.0,18.0
4,20251029,0700,5,맑음 ☀️,70,0,없음,강수없음,0.8,5.0,18.0
5,20251029,0800,6,맑음 ☀️,65,0,없음,강수없음,0.9,5.0,18.0
6,20251029,0900,9,맑음 ☀️,55,0,없음,강수없음,0.9,5.0,18.0
7,20251029,1000,11,맑음 ☀️,50,0,없음,강수없음,0.8,5.0,18.0
8,20251029,1100,13,구름많음 ☁️,55,20,없음,강수없음,1.2,5.0,18.0
9,20251029,1200,15,구름많음 ☁️,55,20,없음,강수없음,1.6,5.0,18.0
